# Post Hoc Analysis Before 11/5/2023

In [642]:
import joblib
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pickle
import time
from tkinter import *
import tkinter as tk
from tkinter import filedialog
from tqdm import tqdm
from tqdm import notebook
import itertools
import math
import seaborn as sns
import statistics 
from collections import OrderedDict
import csv

In [643]:
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)
infiles = filedialog.askopenfilename(multiple=True, title='load posthoc-predictions.npy')

%gui tk

In [644]:
infiles

''

In [645]:
Random_List = [] #creates a array known as random list 
for f in range(len(infiles)):
    arr = np.load(infiles[f])
    Random_List.extend(arr) 
print(Random_List)

[]


# Total Time of Video in deciseconds

In [646]:
duration_in_deciseconds = len(Random_List)
duration_in_minutes = duration_in_deciseconds / 600
duration_in_frames = duration_in_deciseconds * 3

print('This video is', duration_in_deciseconds, 'deciseconds, which is', duration_in_minutes, 'minutes, which is', duration_in_frames, 'frames')
print(Random_List)

This video is 0 deciseconds, which is 0.0 minutes, which is 0 frames
[]


## Creating Array

## Time Integration (deciseconds) 

In [647]:
BehaviorTime = []

for index, value in enumerate(Random_List):
    BehaviorTime.append((value, index))

In [648]:
print(BehaviorTime)

[]


### Ensures you only analyze expected length of  Session 


length_of_session = (int(input("How long was the session in minutes?"))) * 600
print(length_of_session)
BehaviorTime = BehaviorTime[:(length_of_session+1)]

### How Long is the Original Video File 

In [649]:
print('This video is ', len(BehaviorTime), ' deciseconds, which is', (len(BehaviorTime)/600), 'minutes' )
print()
#print(BehaviorTime)

This video is  0  deciseconds, which is 0.0 minutes



## Add Duration, Frames, Stim Count, Start Time, End Time

In [650]:
for z in BehaviorTime:
    behavior = z[0]
    time= z[1]
behavior_type_and_duration = []
current_behavior = BehaviorTime[0][0]
current_time = BehaviorTime[0][1]  
duration = 1
stim_count=0
frames = 0

for behavior, time in BehaviorTime:
    start_time= current_time
    if behavior == current_behavior:
        duration += 1
      
    else:
        if duration==1:
            end_time=start_time+1
        else:
            end_time= start_time + duration
        frames= duration*3
        stim_count= round(duration/3)
        behavior_type_and_duration.append((current_behavior, duration,frames,stim_count,start_time, end_time))
        current_behavior = behavior
        current_time = time
        start_time= time
        duration = 1
        end_time= time+1
        
# Append the final (current_behavior, current_time, count) tuple outside the loop
behavior_type_and_duration.append((current_behavior, duration, frames, stim_count, start_time, end_time))

# Make a copy of the original behavior_type_and_duration list
behaviortdt= list(behavior_type_and_duration)

IndexError: list index out of range

In [ ]:
print(behaviortdt)
#print(len(behaviortdt))

In [ ]:
#length_of_session = (int(input("How long was the session in minutes?"))) * 600 
#print(length_of_session) 
behaviortdt = [tup for tup in behaviortdt if tup[4] <= 39000]
behaviortdt=[tup for tup in behaviortdt if tup[1]<= 5]
count_greater_than_5 = sum(1 for tup in behaviortdt if tup[1] > 5)

In [ ]:
print('This video is ', behaviortdt[-1][-1], ' deciseconds, which is', (behaviortdt[-1][-1]/600), 'minutes' )
print("Number of tuples with a second element greater than 5 :", count_greater_than_5)
print()
print(behaviortdt)

## Add Bout Count to the Array 

In [ ]:
enumerated_behaviortdt = []
behavior_index = {}  # Dictionary to store behavior and its corresponding index

for behavior, duration, frames, stim_count, start_time, end_time in behaviortdt:
    if behavior not in behavior_index:
        behavior_index[behavior] = 1
    else:
        behavior_index[behavior] += 1
    
    bout = behavior_index[behavior]
    enumerated_behaviortdt.append((behavior,stim_count,frames, bout, duration,start_time,end_time))

print(enumerated_behaviortdt)

# Renaming enumerated_behaviortdt

In [ ]:
#BCBDT = []

# Calculate durations and update BCBDT
#for behavior, duration, frames, bout, start_time, end_time in enumerated_behaviortdt:
    #stim_count = round(duration / 3)
    #if stim_count < 1:
        #stim_count = 0
    #if stim_count > 0:
        #BCBDT.append((behavior, duration, frames, bout, stim_count, start_time, end_time))

# Print the modified BCBDT
#print("Modified BCBDT:", BCBDT)

# Assign enumerated_behaviortdt to BCBDST
BCBDST = enumerated_behaviortdt


## Naming Behaviors

In [ ]:
behavior_labels = {
    0.0: "Right Turns",
    1.0: "Left Turns",
    2.0: "Locomotion",
    3.0: "Face Groom",
    4.0: "All Other Groom",
    5.0: "All Other",
}

bdbt_dict = {}  # Dictionary to store lists of behaviors

for behavior,stim_count,frames, bout, duration,start_time,end_time in BCBDST:
    behavior_label = behavior_labels.get(behavior, "Unknown Behavior")
    if behavior_label not in bdbt_dict:
        bdbt_dict[behavior_label] = []
    bdbt_dict[behavior_label].append((behavior, stim_count, frames, bout, duration, start_time,end_time))


In [ ]:


print(bdbt_dict)

In [ ]:
keys_list = bdbt_dict.keys()
print(keys_list)

## Stim Count and Average Duration 

In [ ]:
from tabulate import tabulate
stimcount_by_key = {}

def round_float(value):
    return round(value, 3)

for key, tuples_list in bdbt_dict.items():
    stimcount = [tuple_item[1] for tuple_item in tuples_list]
    sum_of_stimcount = sum(stimcount)
    stimcount_by_key[key] = sum_of_stimcount

table_data = [(key, round_float(sum_value)) for key, sum_value in stimcount_by_key.items()]
headers = ["Key", "Total Number of Stimulations"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

average_duration_by_key = {}

for key, tuples_list in bdbt_dict.items():
    durations = [tuple_item[4] for tuple_item in tuples_list]
    average_duration = ((sum(durations) / len(durations))/10)
    average_duration_by_key[key] = round_float(average_duration)

table_data = [(key, avg_duration) for key, avg_duration in average_duration_by_key.items()]
headers = ["Key", "Average Duration(seconds)"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

stim_block_stim_count_by_key = {}

for key, tuples_list in bdbt_dict.items():
    sum_of_stimcount = 0
    for tuple_item in tuples_list:
        if tuple_item[5]> 3000 and tuple_item[6] <= 21000:  
            sum_of_stimcount += tuple_item[1]
    stim_block_stim_count_by_key[key] = sum_of_stimcount

table_data = [(key, round_float(sum_value)) for key, sum_value in stim_block_stim_count_by_key.items()]
headers = ["Key", "Total Stimulations during 30-Min Stim Block"]
#for (key, value) in stim_block_stim_count_by_key:
    #print(key)
    #print(value)
table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

average_duration_30min_by_key = {}

for key, tuples_list in bdbt_dict.items():
    total_duration = 0  # Initialize the total duration to 0
    count = 0  # Initialize a count variable to track the number of valid tuples
    
    for tuple_item in tuples_list:
        if tuple_item[5] > 3000 and tuple_item[6] <= 21000:  # Check the conditions
            total_duration += tuple_item[4]  # Add the duration to the total
            count += 1  # Increment the count
            
    if count > 0:
        average_duration = round(((total_duration / count)/10), 2)  # Calculate average and round it
    else:
        average_duration = 0  # Set average to 0 if there are no valid tuples
    
    average_duration_30min_by_key[key] = average_duration

# Printing the results in a tabular format
table_data = [(key, avg_duration) for key, avg_duration in average_duration_30min_by_key.items()]
headers = ["Key", "Average Duration During 30 Min Stim Block (seconds)"]

from tabulate import tabulate

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)




##  Number of Bouts

In [ ]:
bout_by_key = {}

for key, tuples_list in bdbt_dict.items():
    bouts = [tuple_item[3] for tuple_item in tuples_list]
    sum_of_bouts = len(bouts)
    bout_by_key[key] = sum_of_bouts

table_data = [(key, sum_value) for key, sum_value in bout_by_key.items()]
headers = ["Key", "Total Bouts"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

stim_block_bout_count_by_key = {}

for key, tuples_list in bdbt_dict.items():
    bout_count = sum(1 for tuple_item in tuples_list if 3000 < tuple_item[5] and tuple_item[6] <= 21000)
    stim_block_bout_count_by_key[key] = bout_count

table_data = [(key, count) for key, count in stim_block_bout_count_by_key.items()]
headers = ["Key", "30 minute Stim Block Bout Count"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

In [ ]:
import re
for file_path in infiles: # if taking from Z Drive 
    folders = (os.path.dirname(file_path)).split('/')
    print(folders)
    print(len(folders))
    if len(folders) >= 4:  # Check if there are at least 4 elements in the list
        folder_name = folders[-1].lower()
        print(folder_name)
        fol = re.split(r'[-\s]+', folder_name)

        if len(fol) >= 2:  # Check if there are at least 2 elements in the 'fol' list
            if fol[1] == 'face':
                # Merge 'face' and 'groom' into a single folder
                fol[1] = 'face groom'
                fol.pop(2)

            print(fol)
            print(len(fol))
        else:
            print("Folder name does not contain at least two elements.")
    else:
        print("Not enough elements in the folders list.")

## Session Type 

In [ ]:
trial_type = "Stim Session"
sham_session = "n"  # Initialize sham_session to "n" by default

# Check if either "Sham" or 'sham' is in the string 'fol'
if "Sham" in fol or 'sham' in fol:
    sham_session = "y"

# Check if sham_session (converted to lowercase) is "y"
if sham_session.lower() == "y":
    trial_type = "Sham Session"

print(sham_session)
print(trial_type)

## Behavior of Interest

In [ ]:
behavior_mapping = {
    0: "Right Turn ",
    1: "Left Turn ",
    2: "Locomotion ",
    3: "Face Groom ",
    4: "All Other Groom ",
    5: "All Other "
}

print("Please select the behavior of interest:")
for key, value in behavior_mapping.items():
    print(f"{key}: {value}")
# Assuming fol[1] contains the behavior label
fol_label = fol[1].lower()  # Convert to lowercase for case-insensitive comparison

# Check if the fol_label is "rt" or "RT" and set behavior_of_interest_1 accordingly
if fol_label == 'rt':
    behavior_of_interest_1 = 0
elif fol_label =='locomotion':
    behavior_of_interest_1 = 2
elif fol_label == 'left turn':
    behavior_of_interest_1=1
elif fol_label=='face groom':
    behavior_of_interest_1=3
elif fol_label== 'all other groom':
    behavior_of_interest_1=4
elif fol_label=='all other':
    behavior_of_interest_1=5
    

behavior_name_2 = behavior_mapping.get(behavior_of_interest_1, "Unknown")

print("Selected behavior of interest:", behavior_name_2)


## Type of Fiber Connection

In [ ]:
if len(fol)>=3:
    print(len(fol))
    fol_label = fol[2].lower() 
    if fol_label=='bilateral':
        type_of_fiber_connection = 'Bilateral'
    elif fol_label=='contralateral'or fol_label=='contra':
        type_of_fiber_connection = 'Contralateral'
    elif fol_label=='ipsilateral'or fol_label=='ipsi':
        type_of_fiber_connection = 'Ipsilateral'
    else:
        type_of_fiber_connection = 'Bilateral'
else:
    type_of_fiber_connection = 'Bilateral'


#selection = input("Please enter the type of fiber connection for the trial (B for Bilateral, I for Ipsilateral, C for Contralateral): ")


print("Selected type of fiber connection:", type_of_fiber_connection)


## Creating CSV

In [ ]:
rounded_dict = {}

for key, value_list in bdbt_dict.items():
    rounded_values = [
        (
            round(v[0]),     # Rounding and converting to integer
            v[1], v[2], v[3], v[4],
            round(v[5], 2),  # Rounding the sixth float value
            round(v[6], 2)   # Rounding the seventh float value
        )
        for v in value_list
    ]
    rounded_dict[key] = rounded_values

#print(rounded_dict)

bdbt_dict=rounded_dict
#print(bdbt_dict)

# Convert values to JSON strings

import json
json_data = {key: json.dumps(value) for key, value in bdbt_dict.items()}
bdbt_dict=json_data 
#print(bdbt_dict)

In [ ]:
# Iterate through each file in infiles
for file_path in infiles: # if taking from K Drive on home machine
    folders = (os.path.dirname(file_path)).split('/')
    #print(folders)
    # Extract the mouse name from the file path
    mouse_name = folders[7].split('-')[0]
    # Extract the genotype from the file path
    genotype = folders[7].split('-')[1]
    # Extract the date from the file path
    date = folders[-1].split('-')[0]
#for file_path in infiles: # if taking from Z Drive 
    #folders = (os.path.dirname(file_path)).split('/')
    #print(folders)
    # Extract the mouse name from the file path
    #mouse_name = folders[4].split('-')[0]
    # Extract the genotype from the file path
    #genotype = folders[4].split('-')[1]
    # Extract the date from the file path
    #date = folders[-1].split('-')[0]
#for file_path in infiles:   #if taking files from D drive
    #folders = (os.path.dirname(file_path)).split('/')
    #print(folders)
    # Extract the mouse name from the file path
    #mouse_name = folders[6].split('-')[0]
    # Extract the genotype from the file path
    #genotype = folders[6].split('-')[1]
    # Extract the date from the file path
    #date = folders[-1].split('-')[0]    

In [ ]:
bdbt_dict.keys()

In [ ]:
filtered_df=[]
session_number=0
#If on home machine
#db_file_path = "/Users/kaycei/K on Server (NoMachine)/KayCei/Naturalistic_cleaned.pkl"
#If on Lab Computer
db_file_path="Z:/KayCei/Vik practice/VikPracticeDatabase.pkl"
 # Check if the pickle file exists
if os.path.exists(db_file_path):
    # If the file exists, load the existing DataFrame
    with open(db_file_path, 'rb') as f:
        df = pickle.load(f)
else:
    # If the file doesn't exist, create a new DataFrame
    df = pd.DataFrame(columns=["Mouse","Genotype","Date","Selected Behavior","Session Type","Session Number","Fiber Connection",
                               "Right Turn Array","Total Right Turn Bout Count","Average Duration of Right Turn(seconds)",
                               "Total Right Turn Bouts in Stim Block","Right Turn Average Duration in Stim Block(seconds)",
                               "Left Turn Array","Total Left Turn Bout Count","Average Duration of Left Turn(seconds)",
                               "Total Left Turn Bouts in Stim Block","Left Turn Average Duration in Stim Block(seconds)",
                               "Locomotion Array","Total Locomotion Bout Count","Average Duration of Locomotion(seconds)",
                               "Total Locomotion Bouts in Stim Block","Locomotion Average Duration in Stim Block(seconds)",
                               "Face Groom Array","Total Face Groom Bout Count","Average Duration of Face Groom(seconds)",
                               "Total Face Groom Bouts in Stim Block","Face Groom Average Duration in Stim Block(seconds)",
                               "All Other Groom Array","Total All Other Groom Bout Count","Average Duration of All Other Groom(seconds)",
                               "Total All Other Groom Bouts in Stim Block","All Other Groom Average Duration in Stim Block(seconds)",
                               "All Other Array","Total All Other Bout Count","Average Duration of All Other(seconds)",
                               "Total All Other Bouts in Stim Block","All Other Average Duration in Stim Block(seconds)",
                              ])
new_animal_data = {
        "Mouse":mouse_name,
        "Genotype":genotype,
        "Date":date,
        "Selected Behavior":behavior_name_2,
        "Session Type":trial_type,  
        "Session Number":session_number,
        "Fiber Connection":type_of_fiber_connection,
        "Right Turn Array":bdbt_dict.get('Right Turns'),
        "Total Right Turn Bout Count":bout_by_key.get('Right Turns'),
        "Average Duration of Right Turn(seconds)":average_duration_by_key.get('Right Turns'),
        "Total Right Turn Bouts in Stim Block":stim_block_bout_count_by_key.get('Right Turns'),
        "Right Turn Average Duration in Stim Block(seconds)":average_duration_30min_by_key.get('Right Turns'),
        "Left Turn Array":bdbt_dict.get('Left Turns'),
        "Total Left Turn Bout Count":bout_by_key.get('Left Turns'),
        "Average Duration of Left Turn(seconds)":average_duration_by_key.get('Left Turns'),
        "Total Left Turn Bouts in Stim Block":stim_block_bout_count_by_key.get('Left Turns'),
        "Left Turn Average Duration in Stim Block(seconds)":average_duration_30min_by_key.get('Left Turns'),
        "Locomotion Array":bdbt_dict.get('Locomotion'),
        "Total Locomotion Bout Count":bout_by_key.get('Locomotion'),
        "Average Duration of Locomotion(seconds)":average_duration_by_key.get('Locomotion'),
        "Total Locomotion Bouts in Stim Block":stim_block_bout_count_by_key.get('Locomotion'),
        "Locomotion Average Duration in Stim Block(seconds)":average_duration_30min_by_key.get('Locomotion'),
        "Face Groom Array":bdbt_dict.get('Face Groom'),
        "Total Face Groom Bout Count":bout_by_key.get('Face Groom'),
        "Average Duration of Face Groom(seconds)":average_duration_by_key.get('Face Groom'),
        "Total Face Groom Bouts in Stim Block":stim_block_bout_count_by_key.get('Face Groom'),
        "Face Groom Average Duration in Stim Block(seconds)":average_duration_30min_by_key.get('Face Groom'),
        "All Other Groom Array":bdbt_dict.get('All Other Groom'),
        "Total All Other Groom Bout Count":bout_by_key.get('All Other Groom'),
        "Average Duration of All Other Groom(seconds)":average_duration_by_key.get('All Other Groom'),
        "Total All Other Groom Bouts in Stim Block":stim_block_bout_count_by_key.get('All Other Groom'),
        "All Other Groom Average Duration in Stim Block(seconds)":average_duration_30min_by_key.get('All Other Groom'),
        "All Other Array":bdbt_dict.get('All Other'),
        "Total All Other Bout Count":bout_by_key.get('All Other'),
        "Average Duration of All Other(seconds)":average_duration_by_key.get('All Other'),
        "Total All Other Bouts in Stim Block":stim_block_bout_count_by_key.get('All Other'),
        "All Other Average Duration in Stim Block(seconds)":average_duration_30min_by_key.get('All Other'),}
 

In [ ]:
import os
import pickle
import pandas as pd

def rearrange_session_numbers(filtered_df):
    # Sort filtered DataFrame by date
    filtered_df.sort_values(by='Date', inplace=True)

    # Assign session numbers based on the sorted order
    filtered_df['Session Number'] = range(1, len(filtered_df) + 1)

    return filtered_df

def add_animal_to_database(db_file_path, new_animal_data, mouse_name, genotype, date, behavior_name_2, trial_type, type_of_fiber_connection,filtered_df):
    # Check if the pickle file exists
    if os.path.exists(db_file_path):
        # If the file exists, load the existing DataFrame
        with open(db_file_path, 'rb') as f:
            df = pickle.load(f)
    else:
        # If the file doesn't exist, create a new DataFrame
        df = pd.DataFrame(columns=["Mouse","Genotype","Date","Selected Behavior","Session Type","Session Number","Fiber Connection",
                                   "Right Turn Array","Total Right Turn Bout Count","Average Duration of Right Turn(seconds)",
                                   "Total Right Turn Bouts in Stim Block","Right Turn Average Duration in Stim Block(seconds)",
                                   "Left Turn Array","Total Left Turn Bout Count","Average Duration of Left Turn(seconds)",
                                   "Total Left Turn Bouts in Stim Block","Left Turn Average Duration in Stim Block(seconds)",
                                   "Locomotion Array","Total Locomotion Bout Count","Average Duration of Locomotion(seconds)",
                                   "Total Locomotion Bouts in Stim Block","Locomotion Average Duration in Stim Block(seconds)",
                                   "Face Groom Array","Total Face Groom Bout Count","Average Duration of Face Groom(seconds)",
                                   "Total Face Groom Bouts in Stim Block","Face Groom Average Duration in Stim Block(seconds)",
                                   "All Other Groom Array","Total All Other Groom Bout Count","Average Duration of All Other Groom(seconds)",
                                   "Total All Other Groom Bouts in Stim Block","All Other Groom Average Duration in Stim Block(seconds)",
                                   "All Other Array","Total All Other Bout Count","Average Duration of All Other(seconds)",
                                   "Total All Other Bouts in Stim Block","All Other Average Duration in Stim Block(seconds)",
                                  ])

    # Check if the combination of mouse_name, genotype, and date already exists in the DataFrame
    existing_animal = df[(df['Mouse'] == mouse_name) & (df['Genotype'] == genotype) & (df['Date'] == date)]
    
    if not existing_animal.empty:
        print("Animal with the same name, genotype, and date already exists. Skipping addition.")
        return
    # Store the original indices of the DataFrame
    original_indices = df.index
    
    # Filter the DataFrame based on 'Animal Name', 'Genotype', and 'Session Type'
    filtered_df = df[(df['Mouse'] == new_animal_data['Mouse']) & 
                 (df['Genotype'] == new_animal_data['Genotype']) & 
                 (df['Session Type'] == new_animal_data['Session Type'])&
                 (df['Selected Behavior'] == new_animal_data['Selected Behavior'])&
                 (df['Fiber Connection'] == new_animal_data['Fiber Connection'])]
    # Print the filtered DataFrame
    #print(filtered_df)

    # If the filtered DataFrame is not empty, set the 'Session Number' to the number of occurrences plus 1
    if not filtered_df.empty:
        session_number = int(filtered_df.shape[0] + 1)
    else:
        # If the combination is new, set 'Session Number' to 1
        session_number = int(1)

    # Append the new animal data to the DataFrame
    new_animal_data['Session Number'] = session_number
    new_row = pd.DataFrame([new_animal_data])
    df = pd.concat([df, new_row], ignore_index=True)

    # Rearrange session numbers based on date only for the filtered DataFrame if it's not empty
    if not filtered_df.empty:
        filtered_df = rearrange_session_numbers(filtered_df)
        # Use the original indices to update the session numbers in the original DataFrame
        df.loc[original_indices.intersection(filtered_df.index), 'Session Number'] = filtered_df['Session Number']
    else:
        print("No data found for rearrangement. Skipping session number rearrangement.")

    # Save the updated DataFrame back to the pickle file
    with open(db_file_path, 'wb') as f:
        pickle.dump(df, f)

    print("New animal added to the database.")


In [ ]:
add_animal_to_database(db_file_path, new_animal_data, mouse_name, genotype, date, behavior_name_2, trial_type, type_of_fiber_connection,filtered_df)

In [ ]:
# Load the data from the PKL file into a DataFrame
data = pd.read_pickle(db_file_path)

# Group the data by unique combinations of values across multiple columns
grouped = data.groupby(['Mouse', 'Genotype', 'Session Type', 'Selected Behavior', 'Fiber Connection'])

# Define a function to sort each group by date and assign session numbers
def assign_session_numbers(group):
    # Sort the group by date
    group_sorted = group.sort_values(by='Date')
    # Assign session numbers based on the order of rows within the sorted group
    group_sorted['Session Number'] = range(1, len(group_sorted) + 1)
    return group_sorted

# Apply the function to each group
sorted_data = grouped.apply(assign_session_numbers)

# Reset the index of sorted_data to ensure correct alignment
sorted_data.reset_index(drop=True, inplace=True)

# Iterate over rows in sorted_data and update corresponding rows in data
for index, row in sorted_data.iterrows():
    # Find matching rows in data based on common columns
    mask = (data['Mouse'] == row['Mouse']) & (data['Genotype'] == row['Genotype']) & \
           (data['Session Type'] == row['Session Type']) & \
           (data['Selected Behavior'] == row['Selected Behavior']) & \
           (data['Fiber Connection'] == row['Fiber Connection']) & \
           (data['Date'] == row['Date'])
    # Update session number in data based on the match
    data.loc[mask, 'Session Number'] = row['Session Number']

# Display the updated data DataFrame
print(data.tail(4))

# Save the updated DataFrame to a new PKL file if needed
data.to_pickle(db_file_path)